# TB Data Acquisition



In [6]:
import sys
import os
from pathlib import Path
import sqlite3
import requests
import pandas as pd
import time


project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) 
                     if (p / 'notebooks' / '01_data_acquisition.ipynb').exists()),
                    Path.cwd())
os.chdir(project_root)

WHO_GHO_API_BASE = "https://ghoapi.azureedge.net/api"
WHO_INDICATORS = {
    "tb_incidence": "TB_e_inc_tbhiv_num",
    "tb_mortality": "TB_e_mort_exc_tbhiv_num",
}

START_YEAR = 2020
END_YEAR = 2025
YEARS = list(range(START_YEAR, END_YEAR + 1))

DATABASE_PATH = str(project_root / "data" / "database" / "tb_data.db")
RAW_DATA_DIR = project_root / "data" / "raw"

print(f"✓ Project: {project_root}")
print(f"✓ Database: {DATABASE_PATH}")


✓ Project: /Users/joshua/datascienceproject/notebooks
✓ Database: /Users/joshua/datascienceproject/notebooks/data/database/tb_data.db


## API Fetching Functions


## Workflow: Create Database Schema


In [7]:
os.makedirs(os.path.dirname(DATABASE_PATH), exist_ok=True)

for attempt in range(5):
    try:
        conn = sqlite3.connect(DATABASE_PATH, timeout=30.0)
        conn.row_factory = sqlite3.Row
        conn.execute("PRAGMA journal_mode=WAL")
        cursor = conn.cursor()
        break
    except sqlite3.OperationalError as e:
        if "locked" in str(e).lower() and attempt < 4:
            time.sleep(1.0 * (attempt + 1))
            continue
        else:
            raise

cursor.execute("""
    CREATE TABLE IF NOT EXISTS country_metadata (
        country_code TEXT PRIMARY KEY,
        region_code TEXT,
        region_name TEXT,
        UNIQUE(country_code)
    )
""")

conn.commit()

for attempt in range(5):
    try:
        cursor.execute("DROP TABLE IF EXISTS tb_data")
        conn.commit()
        break
    except sqlite3.OperationalError as e:
        if "locked" in str(e).lower() and attempt < 4:
            conn.rollback()
            time.sleep(1.0 * (attempt + 1))
            continue
        else:
            raise

cursor.execute("""
    CREATE TABLE tb_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        country_code TEXT NOT NULL,
        year INTEGER NOT NULL,
        tb_incidence_num REAL,
        tb_mortality_num REAL,
        FOREIGN KEY (country_code) REFERENCES country_metadata(country_code),
        UNIQUE(country_code, year)
    )
""")

cursor.execute("CREATE INDEX IF NOT EXISTS idx_tb_country_year ON tb_data(country_code, year)")

conn.commit()
conn.close()
time.sleep(0.5)
print(f"Database schema created successfully at {DATABASE_PATH}")


Database schema created successfully at /Users/joshua/datascienceproject/notebooks/data/database/tb_data.db


In [8]:
datasets = {}

for key, indicator in [('tb_incidence', WHO_INDICATORS['tb_incidence']),
                      ('tb_mortality', WHO_INDICATORS['tb_mortality'])]:
    url = f"{WHO_GHO_API_BASE}/{indicator}"
    response = requests.get(url, timeout=60)
    response.raise_for_status()
    data = response.json()
    
    df = pd.DataFrame(data['value'])
    df = df[df['TimeDim'].isin(YEARS)].copy()
    
    df.rename(columns={
        'SpatialDim': 'country_code', 'TimeDim': 'year', 'NumericValue': 'value',
        'Low': 'value_low', 'High': 'value_high',
        'ParentLocation': 'region_name', 'ParentLocationCode': 'region_code'
    }, inplace=True)
    
    df = df[df.get('SpatialDimType', '') == 'COUNTRY'].copy()
    df = df.dropna(subset=['country_code', 'year'])
    df = df[df['value'] > 0].copy()
    
    datasets[key] = df
    
    RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
    filepath = RAW_DATA_DIR / f'{key}.csv'
    df.to_csv(filepath, index=False)
    print(f"Saved {len(df)} records to {filepath}")
    
    time.sleep(1)

print(f"Fetched {len(datasets)} datasets")


Saved 835 records to /Users/joshua/datascienceproject/notebooks/data/raw/tb_incidence.csv
Saved 884 records to /Users/joshua/datascienceproject/notebooks/data/raw/tb_mortality.csv
Fetched 2 datasets


## Workflow: Load Data into Database


In [9]:
keep_years = YEARS

time.sleep(0.5)

for attempt in range(5):
    try:
        conn = sqlite3.connect(DATABASE_PATH, timeout=30.0)
        conn.row_factory = sqlite3.Row
        conn.execute("PRAGMA journal_mode=WAL")
        cursor = conn.cursor()
        break
    except sqlite3.OperationalError as e:
        if "locked" in str(e).lower() and attempt < 4:
            time.sleep(1.0 * (attempt + 1))
            continue
        else:
            raise

placeholders = ','.join([f':year_{i}' for i in range(len(keep_years))])
params = {f'year_{i}': year for i, year in enumerate(keep_years)}

cursor.execute(f"""
    DELETE FROM tb_data 
    WHERE year NOT IN ({placeholders})
""", params)
deleted = cursor.rowcount

if deleted > 0:
    print(f"Cleaned up {deleted} records outside year range {min(keep_years)}-{max(keep_years)}")

for key in ['tb_incidence', 'tb_mortality']:
    if key not in datasets:
        continue
    
    df = datasets[key].copy()
    df = df[df['year'].isin(YEARS)].copy()
    
    country_df = df[['country_code', 'region_code', 'region_name']].drop_duplicates()
    country_df = country_df.dropna(subset=['country_code'])
    
    for _, row in country_df.iterrows():
        cursor.execute("""
            INSERT OR REPLACE INTO country_metadata (country_code, region_code, region_name)
            VALUES (:country_code, :region_code, :region_name)
        """, {
            'country_code': row['country_code'],
            'region_code': row.get('region_code'),
            'region_name': row.get('region_name')
        })
    
    inserted = 0
    column_name = 'tb_incidence_num' if key == 'tb_incidence' else 'tb_mortality_num'
    
    for _, row in df.iterrows():
        cursor.execute(f"""
            INSERT INTO tb_data 
            (country_code, year, {column_name})
            VALUES (:country_code, :year, :value)
            ON CONFLICT(country_code, year) DO UPDATE SET
                {column_name} = excluded.{column_name}
        """, {
            'country_code': row['country_code'],
            'year': int(row['year']),
            'value': row.get('value')
        })
        inserted += 1
    
    print(f"Loaded {inserted} TB {key} records (years {min(df['year'])}-{max(df['year'])})")

conn.commit()

try:
    cursor.execute("DROP VIEW IF EXISTS unified_tb_data")
except sqlite3.OperationalError:
    pass

try:
    cursor.execute("DROP TABLE IF EXISTS unified_tb_data")
except sqlite3.OperationalError:
    pass

cursor.execute("""
    CREATE TABLE unified_tb_data AS
    SELECT 
        cm.country_code, cm.region_code, cm.region_name,
        tb.year, 
        tb.tb_incidence_num AS number_of_cases,
        tb.tb_mortality_num AS number_of_deaths
    FROM country_metadata cm
    INNER JOIN tb_data tb ON cm.country_code = tb.country_code
    ORDER BY cm.country_code, tb.year
""")

conn.commit()
conn.close()
print("Data acquisition complete")

Loaded 835 TB tb_incidence records (years 2020-2024)
Loaded 884 TB tb_mortality records (years 2020-2024)
Data acquisition complete


## Workflow: Verify Data


In [10]:
import pandas as pd

conn = sqlite3.connect(DATABASE_PATH, timeout=30.0)
conn.row_factory = sqlite3.Row
conn.execute("PRAGMA journal_mode=WAL")

summary = pd.read_sql("""
    SELECT *
    FROM unified_tb_data
    ORDER BY year
""", conn)
conn.close()

print(f"Data Coverage by Year: {len(summary)} total records")
print(summary)

Data Coverage by Year: 885 total records
    country_code region_code            region_name  year  number_of_cases  \
0            AFG         EMR  Eastern Mediterranean  2020             13.0   
1            AGO         AFR                 Africa  2020          15000.0   
2            ALB         EUR                 Europe  2020              4.0   
3            ARE         EMR  Eastern Mediterranean  2020              5.0   
4            ARG         AMR               Americas  2020            730.0   
..           ...         ...                    ...   ...              ...   
880          VUT         WPR        Western Pacific  2024              2.0   
881          YEM         EMR  Eastern Mediterranean  2024            120.0   
882          ZAF         AFR                 Africa  2024         134000.0   
883          ZMB         AFR                 Africa  2024          19000.0   
884          ZWE         AFR                 Africa  2024          20000.0   

     number_of_deaths 